In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@markdown 0.0 instalar dependências
!pip install rtree &> /dev/null
!pip install geopandas &> /dev/null
!pip install osmnx &> /dev/null
# ERRO % pip install libspatialindex
!pip install matplotlib==3.0.2 &> /dev/null

In [4]:
#@title IMPORTS
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import json
import matplotlib.pyplot as plt
from shapely.geometry import shape
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
import time
import pyproj

from shapely.ops import transform



ox.config(log_console=True, use_cache=True)

In [ ]:
#@title  PREPARATION OF DATA

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import json
import matplotlib.pyplot as plt
from shapely.geometry import shape
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
import time
import pyproj

from shapely.ops import transform



ox.config(log_console=True, use_cache=True)

df_master = pd.read_csv('drive/My Drive/Colab Notebooks/express/df_master.csv', sep=';')
def utm_zone(lat,lon):
   return int(32700-round((45+lat)/90,0)*100+round((183+lon)/6,0))

# função de montagem de isócrinas. ((lat,long) -> shapely.Polygon)
def build_iso(ponto, network_type='drive', trip_time=10, travel_speed=20, infill=True):
    #ponto = (cep_interlagoslat, long)
    
    CRS = pyproj.CRS(f"EPSG:{utm_zone(ponto[0], ponto[1])}")
    WSG84 = pyproj.CRS("EPSG:4326");

    G = ox.graph.graph_from_point(center_point=ponto, dist=1500, network_type=network_type, dist_type='bbox')

    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    x, y = gdf_nodes['geometry'].unary_union.centroid.xy
    center_node = ox.get_nearest_node(G, (y[0], x[0]))
    #center_node = ox.get_nearest_node(G, (ponto))
    G = ox.project_graph(G, to_crs=CRS)
    meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
    for u, v, k, data in G.edges(data=True, keys=True):
        data['time'] = data['length'] / meters_per_minute

    
    def make_iso_polys(G, edge_buff=50, node_buff=75, trip_time=10, infill=False):

#           isochrone_polys = []
          subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

          node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
          nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
          nodes_gdf = nodes_gdf.set_index('id')

          edge_lines = []
          for n_fr, n_to in subgraph.edges():
              f = nodes_gdf.loc[n_fr].geometry
              t = nodes_gdf.loc[n_to].geometry
              edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
              edge_lines.append(edge_lookup)

          n = nodes_gdf.buffer(node_buff).geometry
          e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
          all_gs = list(n) + list(e)
          new_iso = gpd.GeoSeries(all_gs).unary_union

          # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
          if infill:
                return Polygon(new_iso.exterior)
#               new_iso = Polygon(new_iso.exterior)
#           isochrone_polys.append(new_iso)
#           return isochrone_polys
          return new_iso

    res = make_iso_polys(G, trip_time=trip_time, infill=infill)

    project = pyproj.Transformer.from_crs(CRS, WSG84, always_xy=True).transform
    poly = transform(project, res)
    return poly
    isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
    fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,
                            node_size=0, bgcolor='k', figsize=(15,15))
    for polygon, fc in zip(isochrone_polys, iso_colors):
        patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
        ax.add_patch(patch)

    plt.show()    


# recebe e itera um DF, retorna o mesmo DF com coluna "ISOCHRON",
def build_iso_from_dataframe(df, col_lat, col_lon, save_intermediate_steps=True, save_at=100, filename="iso_"):
  from tqdm.notebook import tqdm

  new_gdf = []
  save_counter = 0

  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        save_counter += 1
        ponto = (row[col_lat], row[col_lon])
        row['geometry'] = build_iso(ponto)
        
        if(save_intermediate_steps and (save_counter % save_at == 0)):
            gpd.GeoDataFrame(new_gdf).to_file(f"drive/My Drive/Colab Notebooks/express/shapefiles_dataframe/{filename}_iter{save_counter}.shp")
            print(f'salvando {save_counter}')
        #row['geometry'] = row['geometry'].apply(wkt.loads)
        
        new_gdf.append(row)


  if(save_intermediate_steps):
            gpd.GeoDataFrame(new_gdf).to_file(f"drive/My Drive/Colab Notebooks/express/shapefiles_dataframe/{filename}_FINAL.shp")
  return gpd.GeoDataFrame(new_gdf)

#  return isochron

In [ ]:
inicio = 10747 #@param {type: 'number'}
fim =  12000#@param {type: 'number'}

df_slice = df_master.iloc[inicio:fim]

col_lat = 'LAT'
col_lon = 'LONG'

isocrona_bairros = build_iso_from_dataframe(df_slice, col_lat, col_lon, filename=f'iso_{inicio}_{fim}')

salvando 100
salvando 200
salvando 300


In [54]:
def generate_overlay_dataframe (df_iso, df_sc):
    from tqdm import trange
    from shapely import wkt
    
    COLUNAS_SOMA = ['POP_QTY_census', 'RESD_QTY_census', 'HOUSE_QTY_census', 'APTM_QTY_census'] # add colunas aqui
    COLUNAS_MEDIA = ['AVG_INCOME_census'] # add colunas aqui

    results = []
    # df_sc = df_sc[col]
    df_iso = gpd.GeoDataFrame(df_iso, geometry='geometry', crs='EPSG:4674')
    df_sc = gpd.GeoDataFrame(df_sc, geometry=gpd.points_from_xy(df_sc.LONG, df_sc.LAT), crs='EPSG:4674')
    #df_sc =  gpd.GeoDataFrame(df_sc, geometry='geometry', crs='EPSG:4674')
    df_sc1 = df_sc.set_index('CD_GEOCODI')

    for index in trange(df_iso.shape[0]):
        row = df_iso[df_iso.index == index]
        df_tmp = gpd.overlay(df_sc,  row, how="intersection")
        dados = {}
        for c in COLUNAS_SOMA:
            dados[c] = df_tmp[c].sum()
        for c in COLUNAS_MEDIA:
            dados[c] = round(df_tmp[c].mean(), 2)
        dados['CD_GEOCODI'] = row['CD_GEOCODI']
        #dados['geometry'] = df_sc[df_sc['CD_GEOCODI'] == row['CD_GEOCODI']]
        dados['geometry'] = df_sc1.loc[dados['CD_GEOCODI']]['geometry']
        dados['PLNT_CD'] = row['PLNT_CD']
        dados['LAT'] = row['LAT']
        dados['LONG'] = row['LONG']
        dados = gpd.GeoDataFrame(dados, geometry='geometry')
        results.append(dados)
        
    results = gpd.GeoDataFrame(pd.concat(results))
    
    return results


In [6]:
#@title load files
df = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/express/shapefiles_dataframe/iso_387_1000_iter100.shp', dtype={'CD_GEOCODI': 'object'})
df_master = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/express/df_union.csv', sep=';', dtype={'CD_GEOCODI': 'object'})
#rais = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/express/rais.csv')
#df_bairros = pd.merge(bairros, rais, how='left', on='CD_GEOCODI')

In [11]:
df['CD_GEOCODI'] = df['CD_GEOCODI'].astype(int).astype(str)

In [55]:
dataframe_iso = generate_overlay_dataframe(df, df_master)
dataframe_iso.head()

  0%|          | 0/99 [00:00<?, ?it/s]


KeyError: ignored

In [44]:
dataframe_iso.head()

,POP_QTY_census,RESD_QTY_census,HOUSE_QTY_census,APTM_QTY_census,AVG_INCOME_census,CD_GEOCODI,geometry,PLNT_CD,LAT,LONG
0,29382.081415,9532.314116,8692.675483,96.907895,2.47,355030833000119,None,0.0,-23.615764,-46.415107
355030833000119,29382.081415,9532.314116,8692.675483,96.907895,2.47,NaN,POINT (-46.41474 -23.61627),NaN,NaN,NaN
1,21827.077284,7130.360513,6454.570618,58.144737,2.35,355030833000114,None,0.0,-23.626657,-46.412780
355030833000114,21827.077284,7130.360513,6454.570618,58.144737,2.35,NaN,POINT (-46.41333 -23.62518),NaN,NaN,NaN
2,124754.001775,39560.535326,37085.289504,3069.796407,2.86,355030833000088,None,0.0,-23.609674,-46.446783


In [ ]:
latitude = -23.508234
longitude = -46.837325
m = folium.Map (location = [latitude, longitude], zoom_start = 14)


crs = {'init': 'epsg:4326'}
#conc = pd.merge(data, tr_xy_sp, how='inner', on='ID_TRECHO')
#conc = gpd.GeoDataFrame(df,  crs=crs, geometry='geometry')
conc = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4674')
# conc = conc.rename(columns={'TOTAL_GASTOS':'TOTAL_MENSAL_ALIM_HIG_LIMP'})
# conc['PREDICT_MINI_MKT_ALL'] = conc['PREDICT_MINI_MKT_ALL']
# conc['PREDICT_MKT'] = conc['PREDICT_MKT']
# #conc['TOTAL_MENSAL_ALIM_HIG_LIMP'] = conc['TOTAL_MENSAL_ALIM_HIG_LIMP'].round(2)
# #conc['PREDICT_FAT_MEDIO_MENSAL'] = conc['PREDICT_FAT_MEDIO_MENSAL'].round(2)


tooltip=GeoJsonTooltip(
      fields=['PREDICT'],
      localize=True,
      sticky=False,
      labels=True, 
      
      )

style_function = lambda x: {'color': '#00000000','fillOpacity': '0.6', 'weight': 1, 'fillColor': \
            '#FFF7FBFF' if x['properties']["PREDICT"] < 15000 \
       else '#ECE7F2FF' if 15000 < x['properties']["PREDICT"]  < 20000 \
       else '#D0D1E6FF' if 20000 < x['properties']["PREDICT"] < 25000 \
       else '#A6BDDBFF' if 25000 < x['properties']["PREDICT"] < 30000 \
       else '#74A9CFFF' if 30000 < x['properties']["PREDICT"] < 35000 \
       else '#3690C0FF' if 35000 < x['properties']["PREDICT"] < 40000 \
       else '#0570B0FF' if 40000 < x['properties']["PREDICT"] < 45000 \
       else '#045A8DFF' if 45000 < x['properties']["PREDICT"] < 50000 \
       else '#023858FF'}

geoj = folium.GeoJson(conc, tooltip=tooltip, style_function=style_function).add_to(m)